In [ ]:
import pandas as pd
from dune_client.client import DuneClient
import io
import requests
import time
import os

In [ ]:
# Define the base URL and headers RUSLAN
url = "https://api.dune.com/api/v1/query/'INSERT_QUERY_NUMBER'/results" # insert Dune query number
headers = {"X-DUNE-API-KEY": "your_key"}

# Initialize variables
batch_size = 1000
total_records = 1784584
all_rows = []

# Function to fetch data with retry and backoff
def fetch_data_with_retry(url, headers, params, retries=5, backoff_factor=1):
    for attempt in range(retries):
        try:
            response = requests.get(url, headers=headers, params=params, timeout=30)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            print(f"Attempt {attempt+1} failed: {e}")
            if attempt < retries - 1:
                time.sleep(backoff_factor * (2 ** attempt))  # Exponential backoff
            else:
                raise

# Loop through the results using pagination
for offset in range(0, total_records, batch_size):
    params = {"limit": batch_size, "offset": offset}
    try:
        data = fetch_data_with_retry(url, headers, params)
        rows = data['result']['rows']  # Adjust this if the structure is different
        all_rows.extend(rows)
        print(f"Fetched {offset + batch_size} records")
    except Exception as e:
        print(f"Error fetching data at offset {offset}: {e}")
        break  # Exit the loop if there's an error

# Convert the list of dictionaries to a pandas DataFrame
df = pd.DataFrame(all_rows)

# Save the DataFrame to a CSV file if needed
df.to_csv("scrollwallets.csv", index=False)

# Print the DataFrame to verify the results
print(df)
